In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\hp\\Desktop\\DLProject\\Kidney-Disease-Classification-Deep-Learning-Project'

## Entity

In [5]:
import dagshub
dagshub.init(repo_owner='ojim07', repo_name='Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2ce9db35-2010-49ad-b217-ceddaf504f76&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=30e516591a86bb53c3bc5c55c8abf7d844a002bdb3b9f8bb72e408607fa4bf2e




Accessing as ojim07

Initialized MLflow to track repo "ojim07/Kidney-Disease-Classification-Deep-Learning-Project"

Repository ojim07/Kidney-Disease-Classification-Deep-Learning-Project initialized!

os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ojim07/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="ojim07"
os.environ["MLFLOW_TRACKING_PASSWORD"]="75af0220f6f2cfcfc91c11edb408a1319b928f79"

In [18]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [28]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    training_data_path: Path
    all_params:dict
    mlflow_uri: str
    params_image_size:list
    params_batch_size: int
    

## Configuration

In [29]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        model_evaluation_config = ModelEvaluationConfig(
            model_path="artifacts/training/model.h5",
            training_data_path="artifacts/data_ingestion/data",
            mlflow_uri="https://dagshub.com/ojim07/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return model_evaluation_config

In [30]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
class Evaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data_path,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [31]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    evaluation = Evaluation(model_eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-13 12:16:00,409: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-13 12:16:00,414: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-13 12:16:00,416: INFO: common: created directory at: artifacts]
Found 136 images belonging to 2 classes.
9/9 [==============================] - 135s 13s/step - loss: 8.8894 - accuracy: 0.5000
[2024-08-13 12:18:36,004: INFO: common: json file saved at:scores.json]


2024/08/13 12:19:18 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Cmd('git') failed due to: exit code(3221225477)
  cmdline: git version
2024/08/13 12:19:43 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-13 12:20:09,508: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-08-13 12:20:18,929: INFO: builder_impl: Assets written to: C:\Users\hp\AppData\Local\Temp\tmpn5pr3z1x\model\data\model\assets]


2024/08/13 12:22:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\hp\AppData\Local\Temp\tmpn5pr3z1x\model, flavor: tensorflow), fall back to return ['tensorflow==2.12.0']. Set logging level to DEBUG to see the full traceback.
c:\Users\hp\Desktop\DLProject\Kidney-Disease-Classification-Deep-Learning-Project\Kidney\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/08/13 12:25:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
